In [1]:
import langchain_community
import os
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import DeepLake
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA


C:\Users\Johin\anaconda3\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.1.6) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
os.environ["NVIDIA_API_KEY"] = "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"

In [3]:
loader = CSVLoader(file_path='recipes_data.csv',encoding = 'utf-8',
    csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['directions', 'NER', 'source','ingredients','link','site'],
})

In [ ]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
docs = text_splitter.split_documents(documents)

In [4]:
embeddings = SpacyEmbeddings(model_name="en_core_web_sm")

In [5]:
llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-51b-instruct")

In [6]:
db = DeepLake(dataset_path="./my_deeplake/", embedding=embeddings, read_only = True)
# db.add_documents(docs)

Deep Lake Dataset in ./my_deeplake/ already exists, loading from the storage


In [23]:
system_prompt = (
    "You are an assistant that provides recipes based on available ingredients and desired cuisine. "
    "Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

In [24]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
    input_key="question"
)


In [27]:
def find_recipes(ingredients, cuisine):
    input_question = f"Find me a recipe using these ingredients: {ingredients} with cuisine type: {cuisine}."
    answer = qa.invoke({"question": input_question})
    return answer['result']

In [28]:
ingredients_input = "chicken, tomatoes, garlic"
cuisine_input = "Italian"
recipe_response = find_recipes(ingredients_input, cuisine_input)
print(recipe_response)

Based on the ingredients you provided, I found a classic Italian dish that fits your request. Here is a recipe for Pan-Seared Chicken with Tomato and Garlic, also known as "Pollo alla Cacciatora":

**Ingredients:**

* 4 boneless, skinless chicken breasts, cut into 1-inch pieces
* 2 large tomatoes, diced (or 1 can of diced tomatoes)
* 3 cloves of garlic, minced
* 1 tablespoon of olive oil
* Salt and pepper to taste
* 1 teaspoon of dried oregano
* 1/4 cup of white wine (optional)

**Instructions:**

1. Heat the olive oil in a large skillet over medium-high heat. Add the chicken and cook until browned on all sides, about 5-7 minutes. Remove the chicken from the skillet and set aside.
2. Add the minced garlic to the skillet and cook for 1-2 minutes, until fragrant.
3. Add the diced tomatoes to the skillet and cook for 5-7 minutes, until they start to break down and release their juices.
4. Add the chicken back to the skillet and stir in the cooked tomatoes and garlic.
5. Season with salt, 